# qBraid Quantum Jobs: AWS Devices + Amazon Braket

In [1]:
%pip install -q amazon-braket-sdk qbraid-cli


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
!qbraid configure magic

ERROR:qbraid_core.services.environments.magic:Failed to apply magic config at path /opt/.qbraid/environments/qbraid_000000/pyenv/lib/python3.11/site-packages: Failed to write path to /opt/.qbraid/environments/qbraid_000000/pyenv/lib/python3.11/site-packages/qbraid_config.pth.
ERROR:qbraid_core.services.environments.magic:Failed to apply magic config at path /opt/.qbraid/environments/qbraid_000000/pyenv/lib/python3.11/site-packages: Failed to write path to /opt/conda/lib/python3.11/site-packages/qbraid_config.pth.

Successfully configured qBraid IPython magic commands.

You can now use the qBraid-CLI from inside a Jupyter notebook as follows:

        In [1]: %load_ext qbraid_magic

        In [2]: %qbraid



In [3]:
%load_ext qbraid_magic

In [4]:
%qbraid jobs state

Executable: /home/jovyan/.qbraid/environments/amazon_3kbp0j/pyenv/bin/python

Library        State
braket         enabled


In [5]:
# check qBraid credits
%qbraid credits value


qBraid credits remaining: 921.0480


In [6]:
import boto3
from braket.aws import AwsDevice
from braket.circuits import Circuit

# create the Amazon Braket circuit
bell = Circuit().h(0).cnot(0, 1)

print(bell)

T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │


In [7]:
# choose the cloud-based on-demand simulator to run your circuit
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

## UNCOMMENT THE FOLLOWING LINE TO ACCESS ARIA:
# device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1")

In [8]:
# execute the circuit
task = device.run(bell, shots=10)

In [9]:
# display the results
print(task.result().measurement_counts)

Counter({'00': 6, '11': 4})


In [10]:
# list submitted jobs
!qbraid jobs list

Fetching... ⠸ Donerogress

Job ID                                                Submitted                 
Status
aws_sv_sim-tarini-qjob-ioy808aly0zznfhplq9p           2024-10-30T21:51:51.844Z  
INITIALIZED
aws_sv_sim-tarini-qjob-ksxmvsch612niyppv0fx           2024-10-30T21:35:35.061Z  
COMPLETED
aws_sv_sim-tarini-qjob-1azaxqqwzihlee3lu1wc           2024-10-30T21:10:24.593Z  
COMPLETED
aws_sv_sim-tarini-qjob-zzkxx4bigpbtnew5lox9           2024-10-30T19:36:23.010Z  
COMPLETED
aws_iqm_garnet-tarini-qjob-qh0ds4eu8h92o5aemg19       2024-10-30T19:28:49.876Z  
QUEUED
aws_sv_sim-tarini-qjob-bptjf33r5gs2taweahzu           2024-10-30T17:59:51.158Z  
COMPLETED
aws_sv_sim-tarini-qjob-vsybofsshtocytwdjucm           2024-10-29T23:34:52.702Z  
COMPLETED
aws_quera_aquila-tarini-qjob-mjbyoyeg0wfh8xzukr6d     2024-10-29T22:19:55.830Z  
COMPLETED
aws_quera_aquila-tarini-qjob-yz3jhyausycveespa3p7     2024-10-29T22:19:54.986Z  
COMPLETED
aws_quera_aquila-tarini-qjob-bxff53kx4fchzw7io06o     2024-10-29T22

In [11]:
# check remaining qBraid credits
!qbraid credits value

Fetching... ⠧ Donerogress

qBraid credits remaining: 917.2980


In [12]:
client = boto3.client("braket")

In [17]:
# retrieve job data from braket boto3
boto_data = client.search_quantum_tasks(
    filters=[{"name": "quantumTaskArn", "operator": "EQUAL", "values": [task.id]}],
)

job_data = boto_data["quantumTasks"][0]
created = job_data["createdAt"]
ended = job_data["endedAt"]
cost = float(job_data["tags"]["costEstimate"])

print(f"createdAt: {created}")
print(f"endedAt: {ended}")
print(f"costEstimate: {cost} credits ~ ${cost/100}")

ClientError: An error occurred (500) when calling the SearchQuantumTasks operation (reached max retries: 4): 